# Cybersecurity Incident Classification using T5 Transformer

## Project Overview
This notebook demonstrates the evaluation of a fine-tuned T5 transformer model for classifying cybersecurity incidents. The model takes detailed incident descriptions and classifies them into predefined categories and subcategories, helping security teams streamline their incident response process.


## Setup and Dependencies

In [1]:
import pandas as pd
from transformers import T5Tokenizer, T5ForConditionalGeneration, Trainer, TrainingArguments, TrainerCallback
from datasets import Dataset
import pickle
import re
import torch
from tqdm import tqdm
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

## 1. Data Loading and Preprocessing

In [2]:
# Load test dataset
test_dataset = pd.read_csv('./cleaned_dataset/cleaned_test_dataset.csv')

# Load categories and subcategories lists
with open("categories.pkl", "rb") as file:
    categories = pickle.load(file)

with open("subcategories.pkl", "rb") as file:
    subcategories = pickle.load(file)

## 2. Prompt Engineering

In [3]:
def convert_to_prompt(row):
    """
    Converts incident descriptions into structured prompts for the model.
    
    Args:
        row (str): Raw incident description
        
    Returns:
        str: Formatted prompt with instructions and context
    """
    prompt_template = '''
    You are provided with information provided by the victim about a cybersecurity crime, along with exhaustive lists of categories and sub-categories. Your task is to classify the crime into one of the categories and one of the sub-categories based on the provided information.

    Details:
    - **Crime Description**:
    {crime}

    - **Categories**:
    {categories}

    - **Sub-categories**:
    {subcategories}

    **Instructions**:
    1. Based on the provided crime description, identify the most appropriate category and sub-category from the lists provided above.
    2. If the information is insufficient or ambiguous to assign it into a specific category or subcategory, assign up to a maximum of three sub-categories and three categories that are most relevant to the context. Separate multiple categories or sub-categories using `' or '`.
    3. Do not provide explanations or additional text; strictly follow the output format.

    **Output Format**:
    category: <respective_category> subcategory: <respective_subcategory>
    '''

    return prompt_template.format(crime=row, categories=categories, subcategories=subcategories)

## 3. Data Processing Functions

In [4]:
def preprocess_data(data):
    """
    Prepares the dataset for model evaluation.
    
    Args:
        data (pd.DataFrame): Raw dataset with incident descriptions
        
    Returns:
        pd.DataFrame: Processed dataset with input_text and target_text
    """
    df = pd.DataFrame()
    df['input_text'] = data['crimeaditionalinfo'].apply(convert_to_prompt)
    df['target_text'] = data.apply(lambda row: f"category: {row['category']} subcategory: {row['sub_category']}", axis=1)
    return df

# Process test dataset
raw_test_dataset = test_dataset.copy()
prepro_test_dataset = preprocess_data(test_dataset)
test_dataset = Dataset.from_pandas(prepro_test_dataset)

## 4. Model Loading and Inference Setup

In [5]:
# Load fine-tuned model and tokenizer
model = T5ForConditionalGeneration.from_pretrained('./fine_tuned_t5small').to('cuda')
tokenizer = T5Tokenizer.from_pretrained('./fine_tuned_t5small')
model.eval()

def generate_text(inputs):
    """
    Generates predictions for a batch of inputs.
    
    Args:
        inputs (list): List of input texts
        
    Returns:
        list: Generated predictions
    """
    inputs = tokenizer.batch_encode_plus(
        inputs, 
        return_tensors="pt", 
        padding=True, 
        truncation=True, 
        max_length=1500
    )
    inputs = {key: value.to('cuda') for key, value in inputs.items()}

    with torch.no_grad():
        outputs = model.generate(**inputs, max_length=1500)

    return tokenizer.batch_decode(outputs, skip_special_tokens=True)

def extract_details(text):
    """
    Extracts category and subcategory from model output.
    
    Args:
        text (str): Model generated text
        
    Returns:
        tuple: Extracted (category, subcategory)
    """
    pattern = r'category: (.*?) subcategory: (.*)'
    match = re.match(pattern, text)
    if match:
        return tuple(item if item is not None else 'na' for item in match.groups())
    return 'na', 'na'

## 5. Inspecting the generation from the model

In [6]:
test_target = test_dataset['target_text']
test_input = test_dataset['input_text']

In [7]:
print("Input :\n")
print(test_input[15])
print("Expected Target :\n")
print(test_target[15])

Input :


    You are provided with information provided by the victim about a cybersecurity crime, along with exhaustive lists of categories and sub-categories. Your task is to classify the crime into one of the categories and one of the sub-categories based on the provided information.

    Details:
    - **Crime Description**:
    spam message i recieve msg from unwanted number they say you take loan and today is repayment date but i did not take loani recieve text and whatsapp message any time for make repayment of loan but i did not take any loan

    - **Categories**:
    ['Online and Social Media Related Crime', 'Online Financial Fraud', 'Online Gambling  Betting', 'RapeGang Rape RGRSexually Abusive Content', 'Any Other Cyber Crime', 'Cyber Attack/ Dependent Crimes', 'Cryptocurrency Crime', 'Sexually Explicit Act', 'Sexually Obscene material', 'Hacking  Damage to computercomputer system etc', 'Cyber Terrorism', 'Child Pornography CPChild Sexual Abuse Material CSAM', 'Online Cybe

In [9]:
input_text = test_input[15]
input_ids = tokenizer(input_text, return_tensors="pt").input_ids.to("cuda")

outputs = model.generate(input_ids, max_length=1500)
print(tokenizer.decode(outputs[0]))

<pad> category: Online Financial Fraud subcategory: Fraud CallVishing</s>


## 6. Batch Processing and Prediction

In [10]:
# Process test data in batches
batch_size = 128
generated_details = []

for i in tqdm(range(0, len(test_input), batch_size), desc="Processing test data"):
    batch_inputs = test_input[i:i+batch_size]
    generated_texts = generate_text(batch_inputs)
    for generated_text in generated_texts:
        generated_details.append(extract_details(generated_text))

# Convert predictions to DataFrame
test_predicted_df = pd.DataFrame(generated_details, columns=["category", "sub_category"])
test_target_df = raw_test_dataset[["category", "sub_category"]]

Processing test data: 100%|██████████| 222/222 [10:14<00:00,  2.77s/it]


## 7. Model Evaluation

In [11]:
# Create combined predictions for overall metrics
combined_pred = test_predicted_df["category"] + "_" + test_predicted_df["sub_category"]
combined_true = test_target_df["category"] + "_" + test_target_df["sub_category"]

# Calculate metrics
metrics = {
    "Item-level": {
        "Accuracy": accuracy_score(combined_true, combined_pred),
        "Precision": precision_score(combined_true, combined_pred, average="macro", zero_division=0),
        "Recall": recall_score(combined_true, combined_pred, average="macro", zero_division=0),
        "F1-Score": f1_score(combined_true, combined_pred, average="macro", zero_division=0)
    }
}

# Individual column metrics
for column in ["category", "sub_category"]:
    metrics[f"{column.capitalize()}-level"] = {
        "Accuracy": accuracy_score(test_target_df[column], test_predicted_df[column]),
        "Precision": precision_score(test_target_df[column], test_predicted_df[column], average="macro", zero_division=0),
        "Recall": recall_score(test_target_df[column], test_predicted_df[column], average="macro", zero_division=0),
        "F1-Score": f1_score(test_target_df[column], test_predicted_df[column], average="macro", zero_division=0)
    }

# Print metrics
for level, scores in metrics.items():
    print(f"\n{level} Metrics:")
    for metric, value in scores.items():
        print(f"{metric}: {value:.4f}")


Item-level Metrics:
Accuracy: 0.4914
Precision: 0.1154
Recall: 0.0963
F1-Score: 0.0953

Category-level Metrics:
Accuracy: 0.7399
Precision: 0.1912
Recall: 0.1465
F1-Score: 0.1570

Sub_category-level Metrics:
Accuracy: 0.4954
Precision: 0.1324
Recall: 0.1096
F1-Score: 0.1096


### Performance Highlights

1. **Strong Category Classification:** The model achieves a robust 73.99% accuracy in identifying the main category of cybersecurity incidents, which is particularly impressive considering:

    - The complexity of cybersecurity incidents
    - The potential overlap between different categories
    - The use of the smallest variant of the T5 model family

2. **Balanced Performance:** The model maintains consistent performance across subcategories (49.54%) and combined item-level classification (49.14%), showing stable learning across different granularities.
3. **Resource Efficiency:** These results were achieved using T5-small, which is the most lightweight variant of the T5 model family, making it:

    - Computationally efficient
    - Faster in inference time
    - More suitable for deployment in resource-constrained environments